# 좌표상에서 가장 가까운 요소 찾기 (Haver Sine 공식)
예시 : 수도권 지하철역별로 가장 가까운 영화관과 거리 값 표시(km)

In [47]:
import pandas as pd
import numpy as np
from haversine import haversine
import os

In [48]:
# os.chdir('../') #워킹디렉토리에서 한칸 위로 가는 코드(안 써도 됨)

In [49]:
#지하철역 결측치 제거
subway = pd.read_csv('haver sine/지하철 좌표(위경도).csv')
subway.replace([np.inf, -np.inf], np.nan)
subway.dropna(inplace=True)
subway = subway[['name', 'lat', 'lng']]
subway.reset_index(inplace=True)

#영화관 결측치 제거, 영업중인 영화관만 고르기
theater = pd.read_csv('haver sine/영화관 좌표(위경도).csv')
theater.replace([np.inf, -np.inf], np.nan)
theater.dropna(inplace=True)
theater = theater[theater['상세영업상태명']=='영업중']
theater = theater[['사업장명','Lat','Lon']]
theater.reset_index(inplace=True)

In [50]:
print("------지하철역 좌표 데이터------")
print(subway.head())
print("------영화관 좌표 데이터------")
print(theater.head())

------지하철역 좌표 데이터------
   index name        lat         lng
0      0   녹양  37.759380  127.042292
1      1   남영  37.541021  126.971300
2      2   용산  37.529849  126.964561
3      3  노량진  37.514219  126.942454
4      4   대방  37.513342  126.926382
------영화관 좌표 데이터------
   index         사업장명        Lat         Lon
0      0    CGV 방학 4관  37.665184  127.040964
1      1    CGV 방학 6관  37.665184  127.040964
2      2    CGV 방학 5관  37.665184  127.040964
3      3  CGV 방학 4DX관  37.665184  127.040964
4      4    CGV 방학 2관  37.665184  127.040964


In [51]:
#하버사인 거리 계산_ 지하철역 기준 영화관 최단거리
subway_name = []
theater_name = []
between_dist = []
for i in range(0, len(subway)):
    min_distance = 100
    subway_xy = (subway["lat"][i], subway["lng"][i])
    for j in range(0, len(theater)):
        theater_xy = (theater["Lat"][j], theater["Lon"][j])
        distance = haversine(subway_xy, theater_xy, 'km')
        if distance < min_distance:
            min_distance = distance
            near_subway = subway['name'][i]
            near_theater = theater['사업장명'][j]
    subway_name.append(near_subway)
    theater_name.append(near_theater)
    between_dist.append(min_distance)

In [52]:
nearest_theater = np.column_stack((subway_name, theater_name, between_dist))
nearest_theater = pd.DataFrame(nearest_theater) #Data Frame으로 보기 편하게 변경
nearest_theater.columns = ['지하철역명', '영화관명', '거리(km)']

In [53]:
nearest_theater

,지하철역명,영화관명,거리(km)
0,녹양,롯데시네마 노원수락산역 제5관,9.255280393428219
1,남영,씨지브이 용산아이파크몰 제11관,1.3071420273389442
2,용산,씨지브이 용산아이파크몰 제11관,0.2715443160411054
3,노량진,CGV여의도 5관 [CGV아트하우스],2.1346923947867036
4,대방,CGV여의도 5관 [CGV아트하우스],1.3405523288363446
...,...,...,...
692,지식정보단지,떼아뜨르 이동영화,21.504585116185407
693,인천대입구,떼아뜨르 이동영화,21.474036332515723
694,센트럴파크,떼아뜨르 이동영화,21.447187054194494
695,국제업무지구,떼아뜨르 이동영화,21.437183671569333


영화관이 아니라 상영관별로 되어있어서 텍스트에서 '~관'만 깔끔하게 정리하는 작업도 필요할 것 같다.